In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import unidecode
import pandas as pd
import numpy as np
import time

In [2]:
from utils import check_if_titles_match, lower_and_remove_diacritics

Already scraped

artist_name = "Severina"
artist_name = "Mia Dimšić"
artist_name = "Jelena Rozga"

In [3]:
artist_name = "Petar Grašo"
df = pd.read_csv(f"data/{artist_name}_tekstovinet.csv")


In [4]:
df.head()

,Song_ID,Artist,Artist_diskografija,Song_tekstovinet,Views_tekstovinet,Lyrics_tekstovinet,Url_tekstovinet
0,nkaHNg1X_1_0,Petar Grašo,Petar Grašo,92,56927,Otkad nema te tu \r\nsve duzi je dan\r\nsve te...,"https://tekstovi.net/2,221,5620.html"
1,nkaHNg1X_1_1,Petar Grašo,Petar Grašo,Ako se rastanemo,26072,Ne govorim s tobom\r\ni nista nije kao prije\r...,"https://tekstovi.net/2,221,5621.html"
2,nkaHNg1X_1_2,Petar Grašo,Petar Grašo,Ako te pitaju,185913,"Jos su do lani\r\nmoje noci, moji dani \r\nbil...","https://tekstovi.net/2,221,55130.html"
3,nkaHNg1X_1_3,Petar Grašo,Petar Grašo,Baraba,14089,Kazi mi s kim si \r\nnocas cekala zoru\r\nkome...,"https://tekstovi.net/2,221,5622.html"
4,nkaHNg1X_1_4,Petar Grašo,Petar Grašo,Brod ljubavi,27037,"Iznad tihog oceana\r\nbura, nevera, orkana\r\n...","https://tekstovi.net/2,221,5623.html"


In [6]:
df_problematic = pd.DataFrame(columns =  ["Artist"])

In [5]:
def search_google(search_query):
     #search
    driver.find_element(By.NAME, 'q').clear()
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)

In [7]:
def get_lyricstranslate():

    try: driver.find_element(By.PARTIAL_LINK_TEXT, 'Search for English results only').click()
    except: pass
    time.sleep(0.5)

    driver.find_element(By.PARTIAL_LINK_TEXT, 'lyricstranslate.com').send_keys(Keys.CONTROL + Keys.RETURN)
    time.sleep(0.5)
    window_handles = driver.window_handles
    driver.switch_to.window(window_name=window_handles[1])    
    time.sleep(0.5)


    try: driver.find_element(By.XPATH, "//*[contains(text(), 'AGREE')]").click()
    except: pass
    time.sleep(0.5)
    
    try: driver.find_element(By.PARTIAL_LINK_TEXT, 'English').click()
    except: pass
    time.sleep(0.5)

    try: driver.find_element(By.PARTIAL_LINK_TEXT, 'Click to see the original lyrics').click()
    except: pass
    time.sleep(0.5)

    
    try:
        original_lyrics_raw = driver.find_element(By.CLASS_NAME, 'song-node-text')
        original_title = original_lyrics_raw.find_element(By.CLASS_NAME, 'title-h2').text
        par_class = original_lyrics_raw.find_elements(By.CLASS_NAME, 'par')
        original_lyrics = ". ".join([t.text for t in par_class])
        original_lyrics = original_lyrics.replace("*", "")
    except: 
        print("Couldn't find english lyrics")
        original_title = np.nan
        original_lyrics = np.nan

    try:
        eng_lyrics_raw = driver.find_element(By.CLASS_NAME, 'translate-node-text')
        eng_title = eng_lyrics_raw.find_element(By.CLASS_NAME, 'title-h2').text
        par_class = eng_lyrics_raw.find_elements(By.CLASS_NAME, 'par')
        eng_lyrics = ". ".join([t.text for t in par_class])
        eng_lyrics = eng_lyrics.replace("*", "")
    except: 
        print("Couldn't find english lyrics")
        eng_title = np.nan
        eng_lyrics = np.nan

        
    try:
        copyright = original_lyrics_raw.find_element(By.CLASS_NAME, 'copyrighttext').text.split("\n")
        writer_info = [c for c in copyright if "Writer" in c][0]
        if len(writer_info)>0:
            writer = writer_info.split(": ")[1]

    except: 
        print("Couldn't find writer info")
        writer = np.nan


    try:
        song_info = driver.find_element(By.CLASS_NAME, 'song-node-info')
        album_name = song_info.find_element(By.XPATH, "//*[contains(text(), 'Album')]").text
        album_name = album_name.split(": ")[1]
    except: 
        print("Couldn't find album info")
        album_name = np.nan

    
    song_url = driver.current_url
    driver.close()
    driver.switch_to.window(window_name=window_handles[0])
    
    
    return writer, original_title, eng_title, album_name, original_lyrics, eng_lyrics, song_url

In [8]:
ser = Service("chromedriver.exe")
op = webdriver.ChromeOptions()
op.add_argument("--lang=en")  
driver = webdriver.Chrome(service=ser, options=op)
driver.get("https://google.com")
time.sleep(0.5)
try: driver.find_element(By.XPATH, "//*[contains(text(), 'Aceito')]").click()
except: driver.find_element(By.XPATH, "//*[contains(text(), 'I agree')]").click()
time.sleep(0.5)
try: driver.find_element(By.XPATH, "//*[contains(text(), 'English')]").click()
except: pass
time.sleep(0.5)

df_new = df.copy()##[16:17]
for row in df_new.iterrows():
    time.sleep(1)
    artist_name = row[1].Artist#Diskografija
    song_name = row[1].Song_tekstovinet
    try: song_name = song_name[0:song_name.index("(")]
    except:pass
    
    try:
        search_google(lower_and_remove_diacritics(artist_name+" "+song_name+ " english lyricstranslate"))
        writer, original_title, eng_title, album_name, original_lyrics, eng_lyrics, song_url = get_lyricstranslate()

        df.loc[row[0], "Writer_lyricstranslate"] = writer 
        df.loc[row[0], "Song_lyricstranslate"] = original_title
        df.loc[row[0], "EngTitle_lyricstranslate"] = eng_title
        df.loc[row[0], "Album_lyricstranslate"] = album_name
        df.loc[row[0], "Lyrics_lyricstranslate"] = original_lyrics
        df.loc[row[0], "EngLyrics_lyricstranslate"] = eng_lyrics
        df.loc[row[0], "Url_lyricstranslate"] = song_url

        print(f"Succesfully found information for {song_name} from {artist_name}")
    
    except Exception as e:
        df_problematic.loc[row[0], "Song_ID"] = row[1].Song_ID
        df_problematic.loc[row[0], "Artist"] = artist_name
        df_problematic.loc[row[0], "Song_lyricstranslate_run"] = song_name
        df_problematic.loc[row[0], "Error_lyricstranslate"] = e
        print(f"Problem with finding info for {song_name} from {artist_name}")
        print(e)
    print("#######")
    time.sleep(3)
    


Couldn't find writer info
Succesfully found information for 92 from Petar Grašo
#######
Couldn't find album info
Succesfully found information for Ako se rastanemo from Petar Grašo
#######
Couldn't find album info
Succesfully found information for Ako te pitaju from Petar Grašo
#######
Couldn't find writer info
Couldn't find album info
Succesfully found information for Baraba from Petar Grašo
#######
Couldn't find writer info
Couldn't find album info
Succesfully found information for Brod ljubavi from Petar Grašo
#######
Couldn't find writer info
Couldn't find album info
Succesfully found information for Da li iko navija za nas from Petar Grašo
#######
Couldn't find writer info
Couldn't find album info
Succesfully found information for Fritula  from Petar Grašo
#######
Couldn't find writer info
Couldn't find album info
Succesfully found information for Godine ljubavi from Petar Grašo
#######
Couldn't find writer info
Couldn't find album info
Succesfully found information for Gol, bos, 

In [64]:
wrong_rows = check_row_similariy(df, "Lyrics_tekstovinet", "Lyrics_lyricstranslate", threshold=0.05)

In [54]:
##set parameters as nan for wrong rows
for row in wrong_rows:
    df.loc[row, "Writer_lyricstranslate"] = np.nan
    df.loc[row, "Song_lyricstranslate"] = np.nan
    df.loc[row, "EngTitle_lyricstranslate"] = np.nan
    df.loc[row, "Album_lyricstranslate"] = np.nan
    df.loc[row, "Lyrics_lyricstranslate"] = np.nan
    df.loc[row, "EngLyrics_lyricstranslate"] = np.nan
    df.loc[row, "Url_lyricstranslate"] = np.nan

In [55]:
check_if_titles_match(df, "Song_tekstovinet", "Song_lyricstranslate")

,Song_tekstovinet,Song_lyricstranslate
0,92,'92
5,da li iko navija za nas,da li i'ko navija za nas
10,je l' ti reka' 'ko,jel' ti reka 'ko
12,jos uvik slipo virujen,jos uvik slipo virujem
15,ko nam brani,tko nam brani
18,ljubav pozlati sve,ljubav sve pozlati
26,nece nas zauvik bit',nece nas zauvik bit
30,nisan vise ja sa njom,nisam vise ja sa njon
36,sad te se samo rijetko sjetim,sad te se samo rijetko sjeti
46,vera od suvog zlata,vera od suvoga zlata


In [56]:
#check if number of nans is what you would expect
df.Lyrics_lyricstranslate.isnull().sum()

22

In [58]:
len(df)

50

In [57]:
#check if there are some very short lyrics
df.Lyrics_lyricstranslate.dropna().map(lambda x: len(x)<10).sum()

0

In [75]:
df.Lyrics_lyricstranslate

0      Falit ćeš mi, faliti\nali neću žaliti\nsanjala...
1      [Verse 1]\n1\nStvorio te Bog samo mog da te vo...
2      Večeras ja ulazim u klub\n1\nprva mačka, plesn...
3      Ja sam tebi sve na svijetu vjerovala,\nja sam ...
4      U mojoj ulici jutro donose ptice\ni osmijehe n...
                             ...                        
156    Vrela noć je, grad spava\ntramvaj vozi zadnji ...
157    Rakija se liječi vinom,\n1\nMarihuana kokainom...
158    Kada oči otvorim\nti si prvo što mi padne na p...
159    Živila nan jubav!\nViva l'amor!\nDicu će nan d...
160    Zakomplicirao si stvar\nprevario si me\nmožda ...
Name: Lyrics_lyricstranslate, Length: 161, dtype: object

In [59]:
df.to_csv(f"data/{artist_name}_tekstovinet_lyricstranslate.csv", index=False)
df.loc[wrong_rows].to_csv(f"data/{artist_name}_missing.csv", index=False)